In [6]:
import pandas as pd
import ast
import requests
import time

class RecipeImporter:
    def __init__(self, api_base_url='http://localhost:8080/api'):
        self.api_base_url = api_base_url
        self.session = requests.Session()
        self.user_id = None

    def login(self, email, password):
        """Authentification"""
        try:
            url = f"{self.api_base_url}/v1/auth/login"
            response = self.session.post(url, json={'email': email, 'motDePasse': password})
            response.raise_for_status()
            data = response.json()
            token = data.get('token')
            self.user_id = data.get('id') or data.get('userId')
            self.session.headers.update({'Authorization': f'Bearer {token}'})
            print(f"✅ Authentifié (User ID: {self.user_id})")
            return True
        except Exception as e:
            print(f"❌ Échec login: {e}")
            return False

    def _parse_list(self, data):
        """Désormais plus simple car le nouveau scraper formate mieux les listes"""
        if not data or pd.isna(data) or data == "[]" or data == "":
            return []
        if isinstance(data, list): return data
        try:
            # ast.literal_eval convertit les strings "['a', 'b']" en vraies listes []
            return ast.literal_eval(str(data))
        except:
            return []

    def import_recipe(self, row):
        if self.user_id is None: return
        
        # On simplifie : plus besoin de recalculer les minutes ou nettoyer les textes
        # Tout a été fait lors du scraping (votre "v2")
        recette_payload = {
            'titre': str(row.get('titre', 'Sans titre')),
            'description': str(row.get('description', '')),
            'tempsPreparation': int(row.get('temps_preparation', 0)), # Déjà un entier !
            'tempsCuisson': int(row.get('temps_cuisson', 0)),         # Déjà un entier !
            'difficulte': str(row.get('difficulte', 'MOYEN')).upper(),
            'imageUrl': str(row.get('imageUrl', '')),
            'typeRecette': str(row.get('type_recette', 'PLAT')).upper(),
            'cuisine': str(row.get('cuisine', 'FRANÇAISE')).upper(),
            'ingredients': self._parse_list(row.get('ingredients')),
            'etapes': self._parse_list(row.get('etapes'))
        }

        try:
            url = f"{self.api_base_url}/v1/recettes/user/{self.user_id}"
            response = self.session.post(url, json=recette_payload)
            
            if response.status_code in [200, 201]:
                print(f"✓ Recette créée: {recette_payload['titre']}")
                return True
            else:
                # Log plus détaillé pour le débogage
                print(f"⚠️ Erreur ({response.status_code}) sur '{recette_payload['titre']}': {response.text}")
        except Exception as e:
            print(f"❌ Erreur réseau: {e}")
        return False

    def import_all(self, csv_file, max_recipes=None):
        # On charge le nouveau fichier généré par le scraper v2
        try:
            df = pd.read_csv(csv_file)
        except FileNotFoundError:
            print(f"❌ Erreur : Le fichier {csv_file} est introuvable.")
            return

        if max_recipes:
            df = df.head(max_recipes)
            
        print(f"\n📥 Importation de {len(df)} recettes propres vers l'API...")
        success = 0
        
        for idx, row in df.iterrows():
            print(f"[{idx+1}/{len(df)}]", end=" ")
            if self.import_recipe(row.to_dict()):
                success += 1
            # Pas besoin de trop attendre si votre API est locale
            time.sleep(0.02)
        
        print(f"\n✅ Terminé ! {success}/{len(df)} recettes injectées en base.")

if __name__ == "__main__":
    importer = RecipeImporter()
    
    # 1. Login
    if importer.login('valentin2020@gmail.com', 'Password123@'):
        
        # 2. Correction ID si nécessaire
        if importer.user_id is None:
            importer.user_id = 1179
            
        # 3. Importation du fichier v2 (celui avec les colonnes alignées)
        importer.import_all('marmiton_v2.csv', max_recipes=100)

✅ Authentifié (User ID: 2020)

📥 Importation de 100 recettes propres vers l'API...
[1/100] ✓ Recette créée: Pâte à pizza fine
[2/100] ✓ Recette créée: Blanc de poulet aux endives, champignons et lardons
[3/100] ✓ Recette créée: Tartiflette
[4/100] ✓ Recette créée: Filet de poulet au curry
[5/100] ✓ Recette créée: Chili con carne express
[6/100] ✓ Recette créée: Blanquette de veau : recette traditionnelle
[7/100] ✓ Recette créée: Gratin de Courgettes à la viande
[8/100] ✓ Recette créée: Joue de porc à la bière
[9/100] ✓ Recette créée: Pâte à pizza inratable
[10/100] ✓ Recette créée: Boeuf bourguignon : la vraie recette
[11/100] ✓ Recette créée: Mon couscous marocain
[12/100] ✓ Recette créée: Couscous poulet et merguez facile
[13/100] ✓ Recette créée: Poulet au curry lait de coco et noix de cajou
[14/100] ✓ Recette créée: Gratin Dauphinois
[15/100] ✓ Recette créée: Rôti de porc de Dijon
[16/100] ✓ Recette créée: Osso bucco milanaise
[17/100] ✓ Recette créée: Tarte aux poires
[18/100] ✓ R